In [1]:
#Install dependencies

!pip3 install pandas
!pip3 install sklearn
!pip3 install tensorflow_datasets --ignore-installed tensorflow
!pip3 install matplotlib
!pip3 install numpy
!pip3 install tensorflow-hub --ignore-installed tensorflow
!pip3 install tensorflow-text --ignore-installed tensorflow
!pip3 install tf-models-official --ignore-installed tensorflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 1.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.5/503.5 KB 1.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 1.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.0/307.0 KB 1.1 MB/s eta 0:00:00a 0:00:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1309 sha256=3d6130ca333e94171538de41c74c403eb70c4dbf50a157943123fdf214b4a168
  Stored in directory: /root/.cache/pip/wheels/e4/7b/98/b6466d71b8d738a0c547008b9eb39bf8676d1ff6ca4b22af1c
Successfully built sklearn
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 1.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 813.9 kB/s eta 0:00:0000:0100:12
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 KB 852.9 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 1.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 1.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 975.4 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 KB 1.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 KB 912.1 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.2/149.2 KB 1.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.6/156.6 KB 1.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 602.0 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 KB 737.3 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.8/97.8 KB 685.4 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 KB 831.1 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 1.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 KB 870.9 kB/s eta 0:00:00a 0:00:01
  Using cached tensorflow-2.8.0-cp39-cp39-manylinux2010_x86_64.whl (497.6 MB)
  Using cached protobuf-3.20.0-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.whl (1.0 MB)
  Using cached numpy-1.22.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
  Using cached tensorboard-2.8.0-py3-none-any.whl (5.8 MB)
  Using cached keras-2.8.0-py2.py3-none-any.whl (1.4 MB)
  Using cached setuptools-62.1.0-py3-none-any.whl (1.1 MB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached gast-0.5.3-py3-none-any.whl (19 kB)
  Using cached typing_extensions-4.1.1-py3-none-any.whl (26 kB)
  Using cached tf_estimator_nightly-2.8.0.dev2021122109-py2.py3-none-any.whl (462 kB)
  Usi

  Using cached libclang-13.0.0-py2.py3-none-manylinux1_x86_64.whl (14.5 MB)
  Using cached termcolor-1.1.0-py3-none-any.whl
  Using cached wrapt-1.14.0-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (77 kB)
  Using cached numpy-1.22.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
  Using cached typing_extensions-4.1.1-py3-none-any.whl (26 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached keras-2.8.0-py2.py3-none-any.whl (1.4 MB)
  Using cached tensorflow_io_gcs_filesystem-0.24.0-cp39-cp39-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (2.1 MB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Using cached h5py-3.6.0-cp39-cp39-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (4.5 MB)
  Using cached tf_estimator_nightly-2.8.0.dev2021122109-py2.py3-none-any.whl (462 kB)
  Using cached gast-0.5.3-py3-none-any.whl (19 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 KB 423.5 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Using cached tensorflow_hub-0.12.0-py2.py3-none-any.whl (108 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 KB 78.4 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 KB 533.0 kB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.8/99.8 KB 578.8 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 462.6 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.6/237.6 KB 537.7 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 423.0 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py

  Using cached promise-2.3-py3-none-any.whl
  Using cached dill-0.3.4-py2.py3-none-any.whl (86 kB)
  Using cached tensorflow_metadata-1.7.0-py3-none-any.whl (48 kB)
  Using cached googleapis_common_protos-1.56.0-py2.py3-none-any.whl (241 kB)
  Using cached cachetools-5.0.0-py3-none-any.whl (9.1 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
  Using cached importlib_metadata-4.11.3-py3-none-any.whl (18 kB)
  Using cached charset_normalizer-2.0.12-py3-none-any.whl (39 kB)
  Using cached idna-3.3-py3-none-any.whl (61 kB)
  Using cached joblib-1.1.0-py2.py3-none-any.whl (306 kB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 KB 644.2 kB/s eta 0:00:00a 0:00:01
  Using cached zipp-3.8.0-py3-none-any.whl (5.4 kB)
  Using cached oauthlib-3.2.0-py3-none-any.whl (151 kB)
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73049 sha256=aa35a61a4a3d553d553240b5489c2f89adee79d941687eb8a

In [2]:
!pip install jupyterthemes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 1.9 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 KB 1.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 810.0 kB/s eta 0:00:00 0:00:01


In [3]:
!jt -t onedork -T -N

In [11]:
!pip3 uninstall -y tensorflow

In [13]:
import tensorflow as tf
import tensorflow_text as text
print(tf.__version__)
print("Hola")

2.8.0
Hola


In [20]:
!pip3 install tensorflow

     ━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/497.6 MB 1.9 MB/s eta 0:04:18^C
     ━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/497.6 MB 1.9 MB/s eta 0:04:18
ERROR: Operation cancelled by user


In [5]:
!pip3 uninstall tensorflow

Found existing installation: tensorflow 2.8.0
Uninstalling tensorflow-2.8.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.9/dist-packages/tensorflow-2.8.0.dist-info/*
    /usr/local/lib/python3.9/dist-packages/tensorflow/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.9/dist-packages/tensorflow/include/external/local_config_rocm/rocm/rocm/include/amd-dbgapi.h
    /usr/local/lib/python3.9/dist-packages/tensorflow/include/external/local_config_rocm/rocm/rocm/include/amd_comgr.h
    /usr/local/lib/python3.9/dist-packages/tensorflow/include/external/local_config_rocm/rocm/rocm/include/base/backend_manager.hpp
    /usr/local/lib/python3.9/dist-packages/tensorflow/include/external/local_conf

    /usr/local/lib/python3.9/dist-packages/tensorflow/include/external/local_config_rocm/rocm/rocm/include/hsa/amd_hsa_queue.h
    /usr/local/lib/python3.9/dist-packages/tensorflow/include/external/local_config_rocm/rocm/rocm/include/hsa/amd_hsa_signal.h
    /usr/local/lib/python3.9/dist-packages/tensorflow/include/external/local_config_rocm/rocm/rocm/include/hsa/hsa.h
    /usr/local/lib/python3.9/dist-packages/tensorflow/include/external/local_config_rocm/rocm/rocm/include/hsa/hsa_api_trace.h
    /usr/local/lib/python3.9/dist-packages/tensorflow/include/external/local_config_rocm/rocm/rocm/include/hsa/hsa_ext_amd.h
    /usr/local/lib/python3.9/dist-packages/tensorflow/include/external/local_config_rocm/rocm/rocm/include/hsa/hsa_ext_finalize.h
    /usr/local/lib/python3.9/dist-packages/tensorflow/include/external/local_config_rocm/rocm/rocm/include/hsa/hsa_ext_image.h
    /usr/local/lib/python3.9/dist-packages/tensorflow/include/external/local_config_rocm/rocm/rocm/include/hsa/hsa_ven_

    /usr/local/lib/python3.9/dist-packages/tensorflow/include/external/local_config_rocm/rocm/rocm/include/thrust/detail/integer_math.h
    /usr/local/lib/python3.9/dist-packages/tensorflow/include/external/local_config_rocm/rocm/rocm/include/thrust/detail/integer_traits.h
    /usr/local/lib/python3.9/dist-packages/tensorflow/include/external/local_config_rocm/rocm/rocm/include/thrust/detail/internal_functional.h
    /usr/local/lib/python3.9/dist-packages/tensorflow/include/external/local_config_rocm/rocm/rocm/include/thrust/detail/logical.inl
    /usr/local/lib/python3.9/dist-packages/tensorflow/include/external/local_config_rocm/rocm/rocm/include/thrust/detail/malloc_and_free.h
    /usr/local/lib/python3.9/dist-packages/tensorflow/include/external/local_config_rocm/rocm/rocm/include/thrust/detail/memory_algorithms.h
    /usr/local/lib/python3.9/dist-packages/tensorflow/include/external/local_config_rocm/rocm/rocm/include/thrust/detail/memory_wrapper.h
    /usr/local/lib/python3.9/dis

    /usr/local/lib/python3.9/dist-packages/tensorflow/include/external/local_config_rocm/rocm/rocm/include/thrust/system/detail/adl/remove.h
    /usr/local/lib/python3.9/dist-packages/tensorflow/include/external/local_config_rocm/rocm/rocm/include/thrust/system/detail/adl/replace.h
    /usr/local/lib/python3.9/dist-packages/tensorflow/include/external/local_config_rocm/rocm/rocm/include/thrust/system/detail/adl/reverse.h
    /usr/local/lib/python3.9/dist-packages/tensorflow/include/external/local_config_rocm/rocm/rocm/include/thrust/system/detail/adl/scan.h
    /usr/local/lib/python3.9/dist-packages/tensorflow/include/external/local_config_rocm/rocm/rocm/include/thrust/system/detail/adl/scan_by_key.h
    /usr/local/lib/python3.9/dist-packages/tensorflow/include/external/local_config_rocm/rocm/rocm/include/thrust/system/detail/adl/scatter.h
    /usr/local/lib/python3.9/dist-packages/tensorflow/include/external/local_config_rocm/rocm/rocm/include/thrust/system/detail/adl/sequence.h
    /

    /usr/local/lib/python3.9/dist-packages/tensorflow/include/external/local_config_rocm/rocm/rocm/include/thrust/system/tbb/detail/transform_reduce.h
    /usr/local/lib/python3.9/dist-packages/tensorflow/include/external/local_config_rocm/rocm/rocm/include/thrust/system/tbb/detail/transform_scan.h
    /usr/local/lib/python3.9/dist-packages/tensorflow/include/external/local_config_rocm/rocm/rocm/include/thrust/system/tbb/detail/uninitialized_copy.h
    /usr/local/lib/python3.9/dist-packages/tensorflow/include/external/local_config_rocm/rocm/rocm/include/thrust/system/tbb/detail/uninitialized_fill.h
    /usr/local/lib/python3.9/dist-packages/tensorflow/include/external/local_config_rocm/rocm/rocm/include/thrust/system/tbb/detail/unique.h
    /usr/local/lib/python3.9/dist-packages/tensorflow/include/external/local_config_rocm/rocm/rocm/include/thrust/system/tbb/detail/unique.inl
    /usr/local/lib/python3.9/dist-packages/tensorflow/include/external/local_config_rocm/rocm/rocm/include/thr

In [15]:
# from keras import Sequential
import pandas
import sklearn
from sklearn.model_selection import train_test_split

# from transformers import BertTokenizerFast, TFBertForSequenceClassification, TrainingArguments, Trainer

import tensorflow as tf
import tensorflow_hub as hub
# import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

import numpy as np

tf.get_logger().setLevel('ERROR')


### Define Our Constants

In [16]:
DATASETS_PATH = "../data"
# DATASET_FILE_NAME = "hate-speech-labeled.csv"
DATASET_FILE_NAME = "merged-dataset.csv"
# DATASET_FILE_NAME = "80k_dataset/final copy.csv"
TEST_SIZE = 0.2
TRAIN_SIZE = 0.8
DATA_NEED_PREPROCESSING = False
SENTENCE_MAX_LENGTH = 512
TOKENIZER_MODEL_NAME = "bert-base-uncased"
NUM_OF_LABELS = 5

### Reading Dataset

In [17]:
loadedData = pandas.read_csv(DATASETS_PATH + "/" + DATASET_FILE_NAME)
#dtype={'count': np.integer, 'class': np.integer, 'tweet': str}
loadedData["class"].values.astype(int)

loadedData = loadedData.replace('',np.nan)
loadedData = loadedData.dropna()

loadedData

,count,class,tweet
0,0,2.0,!!! RT @mayasolovely: As a woman you shouldn't...
1,40,2.0,""" momma said no pussy cats inside my doghouse """
2,63,2.0,"""@Addicted2Guys: -SimplyAddictedToGuys http://..."
3,66,2.0,"""@AllAboutManFeet: http://t.co/3gzUpfuMev"" woo..."
4,67,2.0,"""@Allyhaaaaa: Lemmie eat a Oreo &amp; do these..."
...,...,...,...
84383,14240,2.0,I'm just...buzzing Nh hee ehh I really want to...
84384,14241,2.0,I liked a @YouTube video https://t.co/SKxJS9Fl...
84385,14242,2.0,#LaserHairRemoval for permanently smooth skin!...
84386,14243,4.0,Monitoring Your Social Media Mentions https://...


### Dataset Analysis

In [18]:
print("Total Size: ", len(loadedData))
print("Test Size: ", len(loadedData) * TEST_SIZE)
print("Train Size: ", len(loadedData) * TRAIN_SIZE)
print("Does the data set have any null values? ", loadedData.isnull().values.any())
print("Does the data set have any duplicate values? ", loadedData.duplicated().any())
print("Does the data have any missing values? ", loadedData.isna().values.any())
# print("Does the data have offensive words? ", loadedData["offensive_language"].any())
print("Number of hate speech tweets: ", (loadedData["class"] == 0).sum())
print("Number of neutral tweets: ", (loadedData["class"] == 2).sum())
# print("Number of offensive tweets: ", (loadedData["class"] == 1).sum())

Total Size:  84335
Test Size:  16867.0
Train Size:  67468.0
Does the data set have any null values?  False
Does the data set have any duplicate values?  True
Does the data have any missing values?  False
Number of hate speech tweets:  1430
Number of neutral tweets:  49082


### Preprocessing Data
(Some data are already pre-processed)

In [19]:
##Preprocessing...
data = loadedData

tweetsText = data.tweet
tweetLabels = data['class']
tweetLabels = tweetLabels.values.astype(int)
#[:29310]

for t in tweetLabels:
    t = int(t)
    if t != 2 and t != 0 and t != 1 and t != 3 and t != 4:
        print(t)
        print(tweetsText[list(tweetLabels).index(t)])

tweetLabels = tf.keras.utils.to_categorical(tweetLabels, NUM_OF_LABELS)
tweetLabels
# tweetLabels = data['hate_speech']

array([[0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.]], dtype=float32)

### Splitting Data

In [20]:
(trainTexts, testTexts, trainLabels, testLabels) = train_test_split(tweetsText, tweetLabels, test_size=TEST_SIZE, train_size=TRAIN_SIZE)

# trainTexts = [str(text) for text in trainTexts]
# testTexts = [str(text) for text in testTexts]
trainLabels
trainTexts

13955        Merry Christmas to all y'all bitches and hoes
24440    RT @Poochda63: I go to take the trash out and ...
74555    A new favorite: Zeus On A Noose ( prod. Cat So...
3407     RT @shakiraevanss: Criticize Amanda for saying...
74559    but on the flip side, i picked up a bag that's...
                               ...                        
50957    "I said, 'Yeah, your site looks like shit.'" I...
2969     RT @T_Mart88: Jaheim, Donell Jones, Lyfe Jenni...
33872    Paulick Report News Alert: Girvin Adds Louisia...
15649    RT @DivaMonRoe2uHoE: Fuck candy. Eat pussy. &#...
79069    Idk how to start this thread there are so many...
Name: tweet, Length: 67468, dtype: object

## Using BERT

### Extracting BERT Encodings

In [21]:
BERT_PREPROCESSOR_URL = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
BERT_ENCODER_URL = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1"

In [22]:
bert_preprocess_model = hub.KerasLayer(BERT_PREPROCESSOR_URL)

2022-04-11 17:32:08.893803: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /openmpi/lib
2022-04-11 17:32:08.893834: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-11 17:32:08.893878: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ispenws-ubuntu): /proc/driver/nvidia/version does not exist
2022-04-11 17:32:08.894076: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### Trying the pro-processor model on a sample text

In [24]:
text_test = ["I am a bad person"]
text_preprocessed = bert_preprocess_model(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')


Keys       : ['input_type_ids', 'input_word_ids', 'input_mask']
Shape      : (1, 128)
Word Ids   : [ 101 1045 2572 1037 2919 2711  102    0    0    0    0    0]
Input Mask : [1 1 1 1 1 1 1 0 0 0 0 0]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


#### Loading Bert Model from Tensorflow Hub

In [25]:
bert_model = hub.KerasLayer(BERT_ENCODER_URL)

#### Example of using Base Bert model (not fine-tuned for our case)

In [26]:
bert_results = bert_model(text_preprocessed)

print(f'Loaded BERT: {BERT_ENCODER_URL}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')


Loaded BERT: https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Pooled Outputs Shape:(1, 512)
Pooled Outputs Values:[ 0.9758005   0.9739895  -0.02925264  0.56059796 -0.00149745  0.9628321
  0.99692595 -0.99832505 -0.58734024 -0.9989166   0.5046092  -0.9547957 ]
Sequence Outputs Shape:(1, 128, 512)
Sequence Outputs Values:[[ 0.05026884 -0.35970068  0.9747762  ... -1.5251129  -0.05629164
  -0.08406009]
 [ 0.35570413  0.52315307  0.5379909  ... -1.4012098  -0.28661966
  -0.6519316 ]
 [-0.24234876  0.3534003  -0.2118063  ... -1.1380774  -1.9675823
  -0.7172764 ]
 ...
 [-0.02793291 -0.3641665  -0.5045563  ... -0.8176338   0.5176668
  -0.01007128]
 [ 0.02250984 -0.2950455  -0.21348913 ... -0.5035719   0.5915532
  -0.32910576]
 [ 0.3237986  -0.775713   -0.21261033 ... -0.52556145  0.7644428
   0.0780424 ]]


## Build Classification Model based on BERT

In [27]:
def build_classification_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  # text_input = tf.keras.layers.Input(shape=(1,), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(BERT_PREPROCESSOR_URL, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(BERT_ENCODER_URL, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  # net = tf.keras.layers.Flatten()(net)
  # net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  net = tf.keras.layers.Dense(NUM_OF_LABELS, activation='softmax', name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [28]:
classifierModel = build_classification_model()
# raw_result = classifierModel(tf.constant("I am a bad person"))
# print(tf.sigmoid(raw_result))

## Model Structure (Layers)

In [29]:
tf.keras.utils.plot_model(classifierModel)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


### Defining our loss function for binary classification

In [30]:
# loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
# metrics = tf.metrics.BinaryAccuracy()

loss = tf.keras.losses.CategoricalCrossentropy()
metrics = tf.metrics.CategoricalAccuracy(name="accuracy")


### Defining our optimizer

In [31]:
trainTextsVariable = tf.Variable(trainTexts)
testTextsVariable = tf.Variable(testTexts)

trainDataset = tf.data.Dataset.from_tensor_slices((trainTexts, trainLabels)).batch(32)
testDataset = tf.data.Dataset.from_tensor_slices((testTexts, testLabels)).batch(32)

# list(trainDataset.as_numpy_iterator())

for text_batch in trainDataset.take(100):
  # for i in range(2):
  print(f'Review: ', text_batch)
  # label = label_batch.numpy()
  # print(f'Label : {label} ({trainLabels[label]})')


Review:  (<tf.Tensor: shape=(32,), dtype=string, numpy=
array([b"Merry Christmas to all y'all bitches and hoes",
       b'RT @Poochda63: I go to take the trash out and a darkskin nigga stole my damn garbage',
       b'A new favorite: Zeus On A Noose ( prod. Cat Soup ) by @svintjvmes_ https://t.co/XrnXX3tog1 on #SoundCloud',
       b"RT @shakiraevanss: Criticize Amanda for saying the n word, sure, but don't make jokes about her sexual assault, don't be trash.",
       b"but on the flip side, i picked up a bag that's making my heart race when i smoke, so it's not helping my anxiety",
       b'UPDATED FROM WEEK 1\n2017 MLB Pick Em Promo Divsion Standings \nFollow link to view and see where u stand\nhttps://t.co/TSYznJNU0S',
       b'@Alexios1201 Depends on what sort of ore you are dealing with assume you are talking about copper  - oxide or sulfide or both ?',
       b'RT @WolfpackAlan: Oreos: First you twist it, then you- aw crap, it broke.',
       b'@ProfOsinbajo please fixed the elect

In [32]:
epochs = 5
steps_per_epoch = tf.data.experimental.cardinality(trainDataset).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
#AdamW Optimizer is best for BERT (Initialy Adam was used with BERT)
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

print("Epochs: ", epochs)
print("Steps per Epoch: ", steps_per_epoch)
print("Number Train Steps: ", num_train_steps)
print("Number Warmup Steps: ", num_warmup_steps)

Epochs:  5
Steps per Epoch:  2109
Number Train Steps:  10545
Number Warmup Steps:  1054


### Compiling our Model using the Defined Loss and Optimizer

In [33]:
classifierModel.compile(loss=loss, optimizer=optimizer, metrics=metrics)

# Training The Model

In [ ]:
print("Training Model using {BERT_ENCODER_URL}")
trainingHistory = classifierModel.fit(trainDataset, validation_data=testDataset, epochs=epochs)


Training Model using {BERT_ENCODER_URL}
Epoch 1/5
2073/2109 [============================>.] - ETA: 1:35 - loss: 0.5554 - accuracy: 0.7853

## Quickly test our Model predictions